In [ ]:
from davinci_still_tool.resolve_init import GetResolve
from pprint import pprint

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
current_timeline = project.GetCurrentTimeline()


def get_subfolder_by_name(subfolder_name: str):
    all_subfolder = root_folder.GetSubFolderList()
    subfolder_dict = {
        subfolder.GetName(): subfolder for subfolder in all_subfolder
    }
    return subfolder_dict.get(subfolder_name, "")


A065R39V: list = (
    get_subfolder_by_name("DAY_015_20230421")
    .GetSubFolderList()[0]  # type: ignore
    .GetSubFolderList()[1]
    .GetClipList()
)

for clip in A065R39V:
    pprint(clip.GetClipProperty())
    break